In [ ]:
(ql:quickload :cl-hamt)

In [ ]:
(use-package :cl-hamt)

# Who is your daddy and what does he do

This notebook contains demo code for the Common Lisp library `cl-hamt`.
`cl-hamt` provides data structures for sets and dictionaries based on hash array-mapped tries.

The implementation of HAMTs in this library is fully persistent.
A persistent collection is never truly modified; rather, when one wishes to add an entry to a collection, a new collection is returned which contains the new element and the old, unmodified collection is preserved.
The new augmented collection, however, shares as much structure as possible with the old collection.

The garbage collector cleans up any old versions of data structures for us if we're not using them anymore.
All told, these persistent collections don't use much more memory than their imperative counterparts.

Since persistent collections are fundamentally immutable and one never makes destructive updates, they can be much easier to reason about and debug.

### Hash sets

Some basic usage -- adding, removing entries -- of the set API is shown below.

In [ ]:
(empty-set)

In [ ]:
(defvar lost-generation
    (set-insert (empty-set)
                "Ernest Hemingway"
                "F. Scott Fitzgerald"
                "Gertrude Stein"))

In [ ]:
(set-lookup lost-generation "F. Scott Fitzgerald")

In [ ]:
(set-lookup lost-generation "Jack Kerouac")

In [ ]:
(set-size lost-generation)

Adding an entry to a set creates a new set; it does not modify the old one.

In [ ]:
(defvar dude-writers (set-remove lost-generation "Gertrude Stein"))

In [ ]:
(set-lookup dude-writers "Gertrude Stein")

In [ ]:
(set-lookup lost-generation "Gertrude Stein")

Reducing over a collection is the key means of performing an operation on all of its elements.

In [ ]:
(set-reduce (lambda (name str)
                    (concatenate 'string str (string #\linefeed) name))
            lost-generation
            "")

Note that HAMTs store the input by hashing it, which does not preserve any natural ordering (e.g. lexicographic) of the entries.